In [100]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, precision_recall_curve
from sklearn.metrics import confusion_matrix, r2_score, mean_squared_error
from collections import Counter

import seaborn as sns

In [188]:
train_df = pd.read_csv('train.csv')
train_df = train_df.drop([' Fwd Header Length.1',' Label'],axis = 1)
train_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225740,61374,61,1,1,6,6,6,6,6.0,0.0,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
225741,61378,72,1,1,6,6,6,6,6.0,0.0,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
225742,61375,75,1,1,6,6,6,6,6.0,0.0,...,0,20,0.0,0.0,0,0,0.0,0.0,0,0
225743,61323,48,2,0,12,0,6,6,6.0,0.0,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0


In [189]:
test_df = pd.read_csv('test_data.csv')
test_df = test_df.drop(['protocol','timestamp','Unnamed: 79'], axis = 1)
test_df

,dst_port,flow_duration,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,fwd_pkt_len_mean,fwd_pkt_len_std,...,fwd_act_data_pkts,fwd_seg_size_min,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min
0,1900,299298,2,0,334,0,167,167,167.0,0.0,...,2,8,0.00,0.00000,0,0,0.0,0.000,0,0
1,80,85297146,11,10,726,660,66,66,66.0,0.0,...,0,20,10198016.88,20298.32698,10221008,10162492,8494075.5,3483466.238,10220878,27630
2,80,84296976,11,10,726,660,66,66,66.0,0.0,...,0,20,10198785.13,20395.84711,10222021,10163041,8394639.8,3640994.384,10222231,27357
3,1900,602936,3,0,501,0,167,167,167.0,0.0,...,3,8,0.00,0.00000,0,0,152405.0,145721.000,298126,6684
4,443,0,1,0,5203,0,5203,5203,5203.0,0.0,...,1,20,0.00,0.00000,0,0,0.0,0.000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8142,1900,19969977,6,0,1002,0,167,167,167.0,0.0,...,6,8,0.00,0.00000,0,0,9830838.5,5620.500,9836459,9825218
8143,1900,3007484,4,0,864,0,216,216,216.0,0.0,...,4,8,0.00,0.00000,0,0,0.0,0.000,0,0
8144,137,1503677,3,0,276,0,92,92,92.0,0.0,...,3,8,0.00,0.00000,0,0,0.0,0.000,0,0
8145,5353,1002673,4,0,280,0,70,70,70.0,0.0,...,4,8,0.00,0.00000,0,0,0.0,0.000,0,0


In [190]:
train_df_list = train_df.columns
test_df_list = test_df.columns
print(len(train_df_list), len(test_df_list))
for i in range(len(train_df)):
    print(train_df_list[i],test_df_list[i])

77 77
 Destination Port dst_port
 Flow Duration flow_duration
 Total Fwd Packets tot_fwd_pkts
 Total Backward Packets tot_bwd_pkts
Total Length of Fwd Packets totlen_fwd_pkts
 Total Length of Bwd Packets totlen_bwd_pkts
 Fwd Packet Length Max fwd_pkt_len_max
 Fwd Packet Length Min fwd_pkt_len_min
 Fwd Packet Length Mean fwd_pkt_len_mean
 Fwd Packet Length Std fwd_pkt_len_std
Bwd Packet Length Max bwd_pkt_len_max
 Bwd Packet Length Min bwd_pkt_len_min
 Bwd Packet Length Mean bwd_pkt_len_mean
 Bwd Packet Length Std bwd_pkt_len_std
Flow Bytes/s flow_byts_s
 Flow Packets/s flow_pkts_s
 Flow IAT Mean flow_iat_mean
 Flow IAT Std flow_iat_std
 Flow IAT Max flow_iat_max
 Flow IAT Min flow_iat_min
Fwd IAT Total fwd_iat_tot
 Fwd IAT Mean fwd_iat_mean
 Fwd IAT Std fwd_iat_std
 Fwd IAT Max fwd_iat_max
 Fwd IAT Min fwd_iat_min
Bwd IAT Total bwd_iat_tot
 Bwd IAT Mean bwd_iat_mean
 Bwd IAT Std bwd_iat_std
 Bwd IAT Max bwd_iat_max
 Bwd IAT Min bwd_iat_min
Fwd PSH Flags fwd_psh_flags
 Bwd PSH Flags bwd

IndexError: index 77 is out of bounds for axis 0 with size 77

In [191]:
train_df = pd.read_csv('train.csv')
train_df = train_df.drop(' Fwd Header Length.1',axis = 1)
train_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225740,61374,61,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225741,61378,72,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225742,61375,75,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
225743,61323,48,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [192]:
le=LabelEncoder()

In [193]:
train_df[' Label']=le.fit_transform(train_df[' Label'])

In [166]:
train_df = train_df.dropna()
train_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225740,61374,61,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
225741,61378,72,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
225742,61375,75,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
225743,61323,48,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0


In [194]:
train_df_x=train_df.drop([' Label'],axis=1)
train_df_y=train_df[' Label']

In [195]:
scaler = StandardScaler()
scaler.fit(train_df_x)
X_transformed = scaler.transform(train_df_x)

ValueError: Input contains infinity or a value too large for dtype('float64').

In [175]:
print("Class distribution: {}".format(Counter(train_df_y))) ## 1: Ddos, 2: Begnign

Class distribution: Counter({1.0: 128025, 0.0: 97686})


In [167]:
TAG_MIN = train_df[train_df.columns].min()
TAG_MAX = train_df[train_df.columns].max()

In [168]:
def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf

In [169]:
def boundary_check(df):
    x = np.array(df, dtype=np.float32)
    return np.any(x > 1.0), np.any(x < 0), np.any(np.isnan(x))

In [173]:
train_df = normalize(train_df[train_df.columns])
train_df = train_df.dropna()
train_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.000013,8.333338e-09,-0.000518,0.000000e+00,3.582796e-10,0.000000e+00,4.398105e-08,0.000003,4.012388e-07,0.0,...,0.007396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000013,8.333345e-09,-0.000518,1.155353e-07,1.791398e-10,2.242725e-13,4.398105e-08,0.000003,4.012388e-07,0.0,...,0.007396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000013,8.333341e-09,-0.000518,1.155353e-07,1.791398e-10,2.242725e-13,4.398105e-08,0.000003,4.012388e-07,0.0,...,0.007396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000011,8.333340e-09,-0.000518,1.155353e-07,1.791398e-10,2.242725e-13,4.398105e-08,0.000003,4.012388e-07,0.0,...,0.007396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000013,8.333338e-09,-0.000518,0.000000e+00,3.582796e-10,0.000000e+00,4.398105e-08,0.000003,4.012388e-07,0.0,...,0.007396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225740,0.000014,8.333342e-09,-0.000518,1.155353e-07,1.791398e-10,2.242725e-13,4.398105e-08,0.000003,4.012388e-07,0.0,...,0.007396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
225741,0.000014,8.333343e-09,-0.000518,1.155353e-07,1.791398e-10,2.242725e-13,4.398105e-08,0.000003,4.012388e-07,0.0,...,0.007396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
225742,0.000014,8.333343e-09,-0.000518,1.155353e-07,1.791398e-10,2.242725e-13,4.398105e-08,0.000003,4.012388e-07,0.0,...,0.007396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
225743,0.000014,8.333341e-09,-0.000518,0.000000e+00,3.582796e-10,0.000000e+00,4.398105e-08,0.000003,4.012388e-07,0.0,...,0.007396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [176]:
boundary_check(train_df)

(False, True, False)

In [177]:
rf=RandomForestClassifier()
rf.fit(train_df_x,train_df_y)
rf_pred=rf.predict(train_df_x)

In [178]:
lr_df=pd.DataFrame()
lr_df['actual']=train_df_y
lr_df['pred']=rf_pred

In [179]:
print(accuracy_score(train_df_y, rf_pred))

0.9995480946874543


In [181]:
confusion_matrix(train_df_y, rf_pred)

array([[ 97682,      4],
       [    98, 127927]], dtype=int64)